# Graph neural network - CaMML course

## Introduction

## Graph theory

## How to do a GNN?

Thanks to this approach we can transform the theory into an useable algorithm.

In [ ]:
# Here we will take some libraries to construct the neural-networks using torch.
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

# Here are some diverse modules and functions necessary.
import random
import time

import matplotlib.pyplot as plt
import numpy as np
# Local library for extracting data.
from utils import MaterialsDataset, collate_fn, plot_sample, scatter_sum, scatter_mean

/home/lorentyfle/test/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data treatment

In [9]:
# To ensure a reproducible results, we put the same random seed.

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# We will also take the matbench data base log_gvrh data set with 10,987 VRH-average shear moduli calculated using DFT. As a starting data base for training and testing the GNN we will create in this section.

dataset = MaterialsDataset(
    "./inputs/bulk-modulus-dataset.json",
    cutoff=4,  # cutoff radius for finding neighbours
    num_gaussians=40,  # number of gaussians in edge embedding
)
# Sample of the data to visualise it.
sample = dataset[10]
# You can visualise one of structure that was sampled.
# Here, we can see the crystal structure in the left and the graph this crystal structure will provide on the right:
plot_sample(sample)

# From here we split the data set into a training and a validation data set.
train_set, valid_set, test_set = random_split(dataset, [0.8, 0.1, 0.1])
print(f'Number of training examples: {len(train_set)}')
print(f'Number of validation examples: {len(valid_set)}')
print(f'Number of testing examples: {len(test_set)}')

Number of training examples: 8790
Number of validation examples: 1099
Number of testing examples: 1098


## Application: Use of GNN to run an MD.